# <div style="text-align: center"><font color='#dc2624' face='微软雅黑'>偏微分方程有限差分定价</font></div>
## <div style="text-align: center"><font color='#dc2624' face='微软雅黑'>雪球自动敲出 (Snowball AutoCallable) 产品</font></div>

## <font color='#dc2624' face='微软雅黑'>目录</font><a name='toc'></a>

### 1. [**<font color='#dc2624' face='微软雅黑'>产品介绍</font>**](#1)
1. [<font color='#2b4750' face='微软雅黑'>卖瓜故事</font>](#1.1)
2. [<font color='#2b4750' face='微软雅黑'>交易证实</font>](#1.2)
3. [<font color='#2b4750' face='微软雅黑'>情景分析</font>](#1.3)
3. [<font color='#2b4750' face='微软雅黑'>产品复制</font>](#1.4)

### 2. [**<font color='#dc2624' face='微软雅黑'>雪球敲出情况</font>**](#2)
1. [<font color='#2b4750' face='微软雅黑'>普通 OTU</font>](#2.1)
2. [<font color='#2b4750' face='微软雅黑'>变种 OTU</font>](#2.2)

### 3. [**<font color='#dc2624' face='微软雅黑'>雪球不敲入不敲出情况</font>**](#3)
1. [<font color='#2b4750' face='微软雅黑'>普通 DOT</font>](#3.1)
2. [<font color='#2b4750' face='微软雅黑'>普通 DNT</font>](#3.2)
3. [<font color='#2b4750' face='微软雅黑'>变种 DOT</font>](#3.3)
4. [<font color='#2b4750' face='微软雅黑'>变种 DNT</font>](#3.4)

### 4. [**<font color='#dc2624' face='微软雅黑'>雪球敲入不敲出情况</font>**](#4)
1. [<font color='#2b4750' face='微软雅黑'>变种 UOP</font>](#4.1)
2. [<font color='#2b4750' face='微软雅黑'>变种 DKOP</font>](#4.2)
3. [<font color='#2b4750' face='微软雅黑'>变种 KIKO</font>](#4.3)

### 5. [**<font color='#dc2624' face='微软雅黑'>定价雪球</font>**](#5)
---

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import scipy.interpolate as spi
import scipy.sparse as sp
import scipy.linalg as sla
from scipy.sparse.linalg import inv
from scipy.sparse.linalg import spsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
mpl.rcParams['font.sans-serif'] = ['Microsoft YaHei']
mpl.rcParams['axes.unicode_minus'] = False

np.random.seed(1031)
dt_hex = '#2B4750'    # dark teal,  RGB = 43,71,80
r_hex = '#DC2624'     # red,        RGB = 220,38,36
g_hex = '#649E7D'     # green,      RGB = 100,158,125
tl_hex = '#45A0A2'    # teal,       RGB = 69,160,162
tn_hex = '#C89F91'    # tan,        RGB = 200,159,145

In [2]:
import cufflinks as cf
cf.go_offline()

# <font color='#dc2624' face='微软雅黑'>1. 产品介绍</font><a name='1'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)
### <font color='#2b4750' face='微软雅黑'>1.1 卖瓜故事</font><a name='1.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

雪球和凤凰结构的产品是券商和信托公司最喜欢推广的产品，其结构错综复杂，名词繁多晦涩，但对其拨茧抽丝后，发现它们本质上就是多个**障碍期权** (barrier option) 的合体。为了接地气，先看一个老王卖瓜的故事：
<font color='#2b4750' face='微软雅黑'>
> 老王是个大瓜农，自己进了一堆瓜。瓜价现在是 1 块钱。

> 老王跟你说：你给我 100 块，我给你打个赌，在接下来一年里

> - 瓜价只要涨到 1 块 1，我给你 120 元，赌局立马结束 (敲出)
> - 瓜价要是跌到 7 毛，你只能拿回 70 块，赌局也是立马结束 (敲入)
> - 瓜价要是一直在 0.7~1.1 元间徘徊呢，只要你持有满一年，还是给你 120 元

> 你要不要跟老王赌？很显然，如果你认为

> - 瓜价能涨到 1 块 1，肯定赌啊，20% 收益爽歪歪
> - 瓜价涨不到 1 块 1，但也跌不到 7 毛，还是会赌，相当于年化 20% 收益

> 当然，瓜价真的跌到 7 毛，你就要亏 30% 了。事实上，只要你觉得瓜价跌不到 7 毛，参加这个赌局都对你有利。现在通货膨胀厉害，你觉得瓜价怎么着也不至于跌到 7 毛，你大概率赚钱啊，那老王不是要亏了？别忘了，老王是个瓜农啊。瓜价涨了，他西瓜卖的才开心，输给你的只是对冲一下啦。</font>

### <font color='#2b4750' face='微软雅黑'>1.2 交易证实</font><a name='1.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

如果长期对某个标的看好，但短期又怕市场变化，可以用雪球结构锁定短期波动。以下标的为中证 500 指数的一个真实雪球结构产品的交易证实 (term sheet) 如下：

<img src="snowball termsheet.PNG" width=60% height=60%>

下节分析了雪球结构五种情景收益情况，只有一种情况会亏损，一种会保本，其他三种会获利。亏损的情景就是下图中的情景 5，运行期间标的跌破一定程度的保护，而且到期也没有涨回来。

该非保本雪球结构图如下：

<img src="snowball payoff.PNG" width=50% height=50%>

若在敲出观察日触发敲出事件，产品自动提前终止并结算年化 20% 的高息票收益。

### <font color='#2b4750' face='微软雅黑'>1.3 情景分析</font><a name='1.3'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

根据上面雪球产品的条款，可**粗分为三种情景**或**细分为 5 种情景**，先展示细分过程以及每种情景对应的收益或损失。

<img src="snowball scenario.PNG" width=90% height=90%>


定义数学符号：

|名称|期初价格|上界|下界|敲出收益(%)|红利收益(%)|标的价格|到期价格|
|---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|符号|$S_0$|$L_h$|$L_l$|$c_1$|$c_2$|$S_t$|$S_T$|


<br>
从定价的角度来讲，上图的五种情景可以合并成三种：

- **情景一**：合并情景 1 和 2

    到期日内观察日标的价格 $S_t$ 超过了上界 <font color='blue'>$L_h$</font>，即发生敲出。这种情况下产品提前结束，<font color='blue'>收益率 = $c_1\%\times\frac{\text{存续月数}}{12}$</font>
  
 
- **情景二**：情景 3

    标的价格 $S_t$ 一直在 <font color='red'>$[L_l, L_h]$</font> 范围内，未发生触碰上下两条边界，<font color='red'>收益率 = $2c_2\%$</font>


- **情景三**：合并情景 4 和 5

    到期日内观察日标的价格 $S_t$ 跌破了 <font color='green'>$L_l$</font>，到期的时候又回到了期初价格 $S_0$ 之上，但是没有触碰 <font color='green'>$L_h$</font>，那么投资者拿回自己的本金。
   
    到期日内观察日标的价格 $S_t$ 跌破了 <font color='green'>$L_l$</font>，而且到期的时候还在期初价格 $S_0$ 之下，但是没有触碰 <font color='green'>$L_h$</font>，那就要承担本金亏损了。
    
    合并起来收益率可写成 <font color='green'>收益率 = $\text{min}\left(\frac{S_T-S_0}{S_0}, 0\right)=-\text{max}\left(\frac{S_0-S_T}{S_0}, 0\right)=-\frac{1}{S_0}\text{max}\left(S_0-S_T, 0\right)$</font>

### <font color='#2b4750' face='微软雅黑'>1.4 产品复制</font><a name='1.4'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

情景一、二、三符合 MECE (Mutually Exclusive Collectively Exhaustive) 原则，即**相互独立，完全穷尽**，分析如下：

    情景一 - 敲出
    情景二 - 没敲入没敲出
    情景三 - 敲入没敲出

那么我们可以对每个情景下的支付进行定价，然后加总得到整个雪球产品的价格。

### <font color='black' face='微软雅黑'>情景一：敲出</font>

到期日内观察日标的价格 $S_t$ 超过了上界 <font color='blue'>$L_h$</font>，即发生敲出。这种情况下产品提前结束，<font color='blue'>收益率 = $c_1\%\times\frac{\text{存续月数}}{12}$</font>

<img src="S1.jpg" width=70% height=70%>

情景一下的支付可由一个变种的**上涨生效触碰期权** (one-touch up, OTU) 来复制，障碍是离散的，而且当触碰每个障碍时获得的返还不一样，公式如下：

<br>
<font color='blue'>
\begin{equation}
V_1 = \text{OTU}\left(R=\left[\frac{c_1\%}{12},\frac{2c_1\%}{12},\cdots,\frac{24c_1\%}{12}\right]\right) 
\end{equation}
</font>

其中 <font color='blue'>$c_1$</font> 是敲出票息。

### <font color='black' face='微软雅黑'>情景二：没敲入没敲出</font>

标的价格 $S_t$ 一直在 <font color='blue'>$[L_l, L_h]$</font> 范围内，未发生触碰上下两条边界，<font color='blue'>收益率 = $2c_2\%$</font>

情景二下的支付就是一个**双边失效触碰期权 (double no-touch, DNT-AW)** 

<br>
<font color='blue'>
\begin{equation}
V_2 = \text{DNT}(R=2c_2\%)
\end{equation}
</font>

其中 <font color='blue'>$c_2$</font> 是红利票息。

### <font color='black' face='微软雅黑'>情景三：敲入没敲出</font>

情景三下的支付就是卖一个本金为 $\frac{1}{S_0}$，行权价格为期初价格 $K=S_0$ 的看跌期权 (注意负号)，<font color='blue'>收益率 = $-\frac{1}{S_0}\text{max}\left(K-S_T, 0\right)$</font>

<img src="S3 A.jpg" width=70% height=70%>

<img src="S3 B.jpg" width=70% height=70%>

情景三的支付可由一个**上涨失效看跌障碍期权 (Up and Out Put, UOP)** 和一个**双边失效看跌障碍期权 (Double Knock-Out Put, DKOP)** 来复制

<br>
<font color='blue'>
\begin{equation}
V_3 = \frac{1}{S_0}\times[\text{DKOP}(K=S_0,L_u,L_l) - \text{UOP}(K=S_0,L_u)]
\end{equation}
</font>

其中 <font color='blue'>$\frac{1}{S_0}$</font> 是 UOP 和 DKOP 的本金。


验证“情景三的支付”和“复制组合的支付”是否一致，先看 $V_3$ 的支付情况：

- **情景一**：只要敲出 (不管是否敲入)，支付为零
- **情景二**：既没敲入也没敲出，支付为零
- **情景三**：但只要敲入而且从未敲出时，到期日支付为 <font color='blue'>$-\frac{1}{S_0}\text{max}\left(K-S_T, 0\right)$</font>

再看 DKOP - UOP 的支付情况：

- **情景一**：只要敲出，UOP 和 DKOP 都敲出，两个支付都为零，因此总支付为零，吻合
- **情景二**：既没敲入也没敲出，UOP 和 DKOP 到期日的支付一样，两者相减总支付为零，吻合
- **情景三**：当价格触碰 DKOP 下边界但没有触碰上边界时，DKOP 敲出支付为零，UOP 保持看跌期权的支付 $\text{max}(S_0 - S_T, 0)$，带着本金两者相减总支付 <font color='blue'>$-\frac{1}{S_0}\text{max}(S_0 - S_T, 0)$</font>，吻合

# <font color='#dc2624' face='微软雅黑'>2. 雪球敲出情况</font><a name='2'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

敲出时候的支付可由一个变种的**上涨生效触碰期权**来复制，要想对变种类型定价，首先要了解如何对普通的**上涨生效触碰期权**定价。

### <font color='#2b4750' face='微软雅黑'>2.1 普通 OTU</font><a name='2.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

对于普通的上涨生效触碰期权 (one-touch up, OTU)：

- 如果返还支付方式为**到期支付** (pay-at-expiry, PaE)，一旦触碰上障碍获得返还 $R$，但是在到期日 $T$ 才能得到，那么在时点 $t_i$ 期权的价值为 $Re^{-r(T-t_i)}$。
- 如果返还支付方式为**即时支付** (pay-at-hit, PaH)，一旦触碰上障碍立刻获得返还 $R$，那么在时点 $t_i$ 期权的价值就为 $R$。

OTU 的终值条件为零，因为在到期日时如果 $S$ 没有触碰障碍，得不到任何返还。用有限差分求解 OTU 首先需要设定其**终值条件** (terminal condition) 和 **边界条件** (boundary conditions)。

<img src="OTU.png" width=50% height=50%>

In [3]:
class SingleOneTouchOption():
    
    def __init__(self, optionFlag, S, T, r, q, sigma, L, R, RFlag):
        self.S = S
        self.L = L
        self.r = r
        self.q = q
        self.T = T
        self.sigma = sigma
        self.R = R
        self.PaE = RFlag
        self.RFlag = 1 if self.PaE else 0
        self.optionFlag = optionFlag
        self.is_down = (optionFlag[2].lower()=='d')

In [4]:
class OT_FD(SingleOneTouchOption):
    
    def __init__(self, optionFlag, S, T, r, q, sigma, L, R, RFlag, Ns, Nt):
        super().__init__(optionFlag, S, T, r, q, sigma, L, R, RFlag)
                
        if self.is_down:
            self.Smin = self.L
            self.Smax = 4*np.maximum(self.S, self.L)
        else:
            self.Smin = 0
            self.Smax = self.L
        
        self.Ns = int(Ns)
        self.Nt = int(Nt)    
        self.dS = (self.Smax-self.Smin)/Ns * 1.0
        self.dt = T/Nt*1.0
        self.Svec = np.linspace(self.Smin, self.Smax, self.Ns+1)
        self.Tvec = np.linspace(0, T, self.Nt+1)
        self.grid = np.zeros(shape=(self.Ns+1, self.Nt+1))
    
    def _set_terminal_condition_(self):
        pass
    
    def _set_boundary_condition_(self):
        tau = self.Tvec[-1] - self.Tvec     
        DF = np.exp(-self.r*tau*self.RFlag)
        
        if self.is_down:
            self.grid[0,  :] = self.R * DF
        else:
            self.grid[-1, :] = self.R * DF
        
        
    def _set_coefficient_(self):
        drift = (self.r-self.q)*self.Svec[1:-1]/self.dS
        diffusion_square = (self.sigma*self.Svec[1:-1]/self.dS)**2
        
        self.l = 0.5*(diffusion_square - drift)
        self.c = -diffusion_square - self.r
        self.u = 0.5*(diffusion_square + drift)
        
    def _solve_(self):
        pass
    
    def _interpolate_(self):
        tck = spi.splrep( self.Svec, self.grid[:,0], k=3 )
        return spi.splev( self.S, tck )
    
    def price(self):
        self._set_terminal_condition_()
        self._set_boundary_condition_()
        self._set_coefficient_()
        self._set_matrix_()
        self._solve_()
        return self._interpolate_()

In [5]:
class OT_CN(OT_FD):

    theta = 0.5
    
    def _set_matrix_(self):
        self.A = sp.diags([self.l[1:], self.c, self.u[:-1]], [-1, 0, 1],  format='csc')
        self.I = sp.eye(self.Ns-1)
        self.M1 = self.I + (1-self.theta)*self.dt*self.A
        self.M2 = self.I - self.theta*self.dt*self.A
    
    def _solve_(self):           
        _, M_lower, M_upper = sla.lu(self.M2.toarray())        
        for j in reversed(np.arange(self.Nt)):
            
            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )

In [8]:
(optionFlag, S, T, r, q, sigma, L, R, RFlag) = ('OTU', 100, 1, 0.025, 0, 0.1, 103, 2, 0)

测试两个 OTU，第一个是 PaH (`RFlag=1`)，第二个是 PaE (`RFlag=0`)，第一个比第二个稍微大一点。

In [9]:
(Ns, Nt) = (500, np.ceil(T*360))

In [10]:
(optionFlag, S, T, r, q, sigma, L, R, RFlag) = ('OTU', 100, 1, 0.025, 0, 0.1, 103, 2, 0)
OTU1 = OT_CN( optionFlag, S, T, r, q, sigma, L, R, RFlag, Ns, Nt )
V_OTU1 = OTU1.price()

OTU2 = OT_CN( optionFlag, S, T, r, q, sigma, L, R, 1, Ns, Nt )
V_OTU2 = OTU2.price()

print( f'OTU PaH = {V_OTU1:.4f}' )
print( f'OTU PaE = {V_OTU2:.4f}' )

OTU PaH = 1.6136
OTU PaE = 1.5818


In [11]:
d = {'OTU PaH':V_OTU1, 'OTU PaE':V_OTU2}
s = pd.Series(d)
s.iplot( xTitle='种类', yTitle='$', mode='lines+markers', size=10, 
         title='OTU PaH Vs PaE', theme='space' )

### <font color='#2b4750' face='微软雅黑'>2.2 变种 OTU</font><a name='2.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

雪球产品第一部分的变种 OTU 和普通 OTU 情况相比，不同点在于

1. 上障碍是离散的，每个月检查是否敲出。
2. 敲出后支付的返根据敲出时间不同而不同，如果在第 3 个月首次敲出，支付 3/12 * 20%；如果在第 18 个月首次敲出，支付 18/12 * 20%。

正因为是离散障碍，上边界不再是 $L$，而是要延申到 $4\times\text{max}(S_0,L)$：

<img src="OTU discrete.PNG" width=50% height=50%>

In [12]:
class SingleOneTouchOptionV():
    
    def __init__(self, optionFlag, S, T, r, q, sigma, L, R, RFlag, freq=None, T_KO=None):
        self.S = S
        self.L = L
        self.r = r
        self.q = q
        self.T = T
        self.sigma = sigma
        self.R = R
        self.PaE = RFlag
        self.RFlag = 1 if self.PaE else 0
        self.optionFlag = optionFlag
        self.is_down = (optionFlag[2].lower()=='d')
        self.freq = 'daily' if freq is None else freq
        self.is_continuous = freq is None
        self.T_KO = np.array([]) if T_KO is None else T_KO

In [13]:
class OTV_FD(SingleOneTouchOptionV):
    
    def __init__(self, optionFlag, S, T, r, q, sigma, L, R, RFlag, Ns, Nt, KO_freq=None, T_KO=None):
        super().__init__(optionFlag, S, T, r, q, sigma, L, R, RFlag, KO_freq, T_KO)
                
        if self.is_down:
            self.Smin = self.L if self.is_continuous else 0 
            self.Smax = 4*np.maximum(self.S, self.L)
        else:
            self.Smin = 0
            self.Smax = self.L if self.is_continuous else 4*np.maximum(self.S, self.L)
            
        self.Ns = int(Ns)
        self.Nt = int(Nt)    
        self.dS = (self.Smax-self.Smin)/Ns * 1.0
        self.dt = T/Nt*1.0
        self.Svec = np.linspace(self.Smin, self.Smax, self.Ns+1)
        self.Tvec = np.linspace(0, T, self.Nt+1)
        self.grid = np.zeros(shape=(self.Ns+1, self.Nt+1))
    
    def _set_terminal_condition_(self):
        pass
    
    def _set_boundary_condition_(self):
        tau = self.Tvec[-1] - self.Tvec
        DF = np.exp(-self.r*tau*self.RFlag)
        
        if (not self.is_continuous) and (not np.isscalar(self.R)): # discrete barrier with multiple rebates
            f = spi.interp1d(self.T_KO, self.R, kind='next', bounds_error=False, fill_value=(self.R[0],self.R[-1]))
            if self.is_down:
                self.grid[0,  :] = f(self.Tvec) * DF
            else:
                self.grid[-1, :] = f(self.Tvec) * DF
        else: # discrete barrier with single rebate, continous barrier
            if self.is_down:
                self.grid[0,  :] = self.R * DF
            else:
                self.grid[-1, :] = self.R * DF
    
    def _set_coefficient_(self):
        drift = (self.r-self.q)*self.Svec[1:-1]/self.dS
        diffusion_square = (self.sigma*self.Svec[1:-1]/self.dS)**2
        
        self.l = 0.5*(diffusion_square - drift)
        self.c = -diffusion_square - self.r
        self.u = 0.5*(diffusion_square + drift)
        
    def _solve_(self):
        pass
    
    def _interpolate_(self):
        tck = spi.splrep( self.Svec, self.grid[:,0], k=3 )
        return spi.splev( self.S, tck )
    
    def price(self):
        self._set_terminal_condition_()
        self._set_boundary_condition_()
        self._set_coefficient_()
        self._set_matrix_()
        self._solve_()
        return self._interpolate_()

In [14]:
class OTV_CN(OTV_FD):

    theta = 0.5
    
    def _set_matrix_(self):
        self.A = sp.diags([self.l[1:], self.c, self.u[:-1]], [-1, 0, 1],  format='csc')
        self.I = sp.eye(self.Ns-1)
        self.M1 = self.I + (1-self.theta)*self.dt*self.A
        self.M2 = self.I - self.theta*self.dt*self.A
    
    def _solve_(self):    
        if self.is_continuous:
            self._solve_continuous_()
        else:
            self._solve_discrete_()
    
    def _solve_discrete_(self):    
        _, M_lower, M_upper = sla.lu(self.M2.toarray())
        
        # find the KO index from T vector
        KO_idx = np.searchsorted(self.Tvec, self.T_KO)
        
        # update U on [Smin, L] if Down KO, update U on [L, Smax] if Up KO
        update_bool_idx = (self.Svec<=self.L) if self.is_down else (self.Svec>=self.L)
        
        tau = self.Tvec[-1] - self.T_KO
        rbt = self.R * np.exp(-self.r*tau*self.RFlag)
        
        for j in reversed(np.arange(self.Nt)):
            
            if j+1 in KO_idx: # Knock-out
                k = np.where(KO_idx==j+1)[0]
                self.grid[update_bool_idx, j+1] = rbt[k]  

            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )
    
    def _solve_continuous_(self):           
        _, M_lower, M_upper = sla.lu(self.M2.toarray())
        
        for j in reversed(np.arange(self.Nt)):
            
            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )

In [15]:
(optionFlag, S, T, r, q, sigma, L, R, RFlag) = ('OTU', 100, 1, 0.025, 0, 0.1, 103, 2, 0)
freq = 'monthly'
T_KO = np.arange(30, 361, 30)/360

In [17]:
OTU_d = OTV_CN( optionFlag, S, T, r, q, sigma, L, R, RFlag, Ns, Nt, freq, T_KO )
V_OTU_d = OTU_d.price()

OTU_c = OTV_CN( optionFlag, S, T, r, q, sigma, L, R, RFlag, Ns, Nt)
V_OTU_c = OTU_c.price()

print( f'离散障碍 OTU = {V_OTU_d:.4f}' )
print( f'连续障碍 OTU = {V_OTU_c:.4f}' )

离散障碍 OTU = 1.4428
连续障碍 OTU = 1.6136


In [18]:
R_vec = 2*R*T_KO
print( f'返还从少到多 = {R_vec}')
print( f'返还从多到少 = {R_vec[::-1]}\n')

OTU = OTV_CN( optionFlag, S, T, r, q, sigma, L, R_vec, RFlag, Ns, Nt, freq, T_KO )
V_OTU_r1 = OTU.price()

OTU = OTV_CN( optionFlag, S, T, r, q, sigma, L, R_vec[::-1], RFlag, Ns, Nt, freq, T_KO )
V_OTU_r2 = OTU.price()

print( f'离散障碍不同返还 OTU = {V_OTU_r1:.4f}' )
print( f'离散障碍不同返还 OTU = {V_OTU_r2:.4f}' )

返还从少到多 = [0.33333333 0.66666667 1.         1.33333333 1.66666667 2.
 2.33333333 2.66666667 3.         3.33333333 3.66666667 4.        ]
返还从多到少 = [4.         3.66666667 3.33333333 3.         2.66666667 2.33333333
 2.         1.66666667 1.33333333 1.         0.66666667 0.33333333]

离散障碍不同返还 OTU = 0.8840
离散障碍不同返还 OTU = 2.2419


In [19]:
d = {'OTU (离散-返还从少到多)':V_OTU_r1, 'OTU (离散)':V_OTU_d, 'OTU (连续)':V_OTU_c, 'OTU (离散-返还从多到少)':V_OTU_r2}
s = pd.Series(d)
s.iplot( xTitle='种类', yTitle='$', mode='lines+markers', size=10, 
         title='不同返还情况', theme='space' )

# <font color='#dc2624' face='微软雅黑'>3. 雪球不敲入不敲出情况</font><a name='3'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

不敲入不敲出时候的支付可由一个变种的**双边失效触碰期权**来复制，要想对变种类型定价，首先要了解如何对普通的**双边失效触碰期权**定价。

### <font color='#2b4750' face='微软雅黑'>3.1 普通 DOT</font><a name='3.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

对于普通的**双边失效触碰期权** (double no-touch, DNT)，返还的支付方式只能为 PaE

- 如果没有触碰上下障碍，则在到期日 $T$ 获得返还 $R$

与 DNT 互为补集的产品是普通的**双边生效触碰期权** (double one-touch, DOT)，因为 DNT 不碰障碍得返还，而 DOT 碰障碍得返还。

- 如果返还支付方式为 PaE

   - 一旦触碰下障碍获得返还 $R_l$，但是在到期日 $T$ 才能得到，那么在时点 $t_i$ 期权的价值为 $R_l e^{-r(T-t_i)}$
   - 一旦触碰上障碍获得返还 $R_h$，但是在到期日 $T$ 才能得到，那么在时点 $t_i$ 期权的价值为 $R_h e^{-r(T-t_i)}$

- 如果返还支付方式为 PaH

   - 一旦触碰下障碍立刻获得返还 $R_l$，那么在时点 $t_i$ 期权的价值就为 $R_l$
   - 一旦触碰上障碍立刻获得返还 $R_h$，那么在时点 $t_i$ 期权的价值就为 $R_h$

根据 DNT 和 DOT 得性质，它们之间的现金平价关系 (cash parity) 如下：

<br>
<font color='blue'>
\begin{equation}
V_{\text{DNT}} + V_{\text{DOT}} = Re^{-rT} 
\end{equation}
</font>

通常我们使用有限差分求解 DOT，然后再用平价关系算出 DNT。

DOT 的终值条件为零，因为在到期日时如果 $S$ 没有触碰障碍，得不到任何返还。用有限差分求解 DOT 首先需要设定其**终值条件** (terminal condition) 和 **边界条件** (boundary conditions)。

<img src="DOT.png" width=50% height=50%>

In [20]:
class DoubleOneTouchOption():
    
    def __init__(self, optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag):
        self.S = S
        self.Ll = Ll
        self.Lh = Lh
        self.r = r
        self.q = q
        self.T = T
        self.sigma = sigma
        self.lR = R[0]
        self.hR = R[1]
        self.lPaE = RFlag[0]
        self.hPaE = RFlag[1]
        self.lRFlag = 1 if self.lPaE else 0
        self.hRFlag = 1 if self.hPaE else 0
        self.optionFlag = optionFlag

In [21]:
class DOT_FD(DoubleOneTouchOption):
    
    def __init__(self, optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag, Ns, Nt):
        super().__init__(optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag)
                
        self.Smin = self.Ll
        self.Smax = self.Lh
        self.Ns = int(Ns)
        self.Nt = int(Nt)    
        self.dS = (self.Smax-self.Smin)/Ns * 1.0
        self.dt = T/Nt*1.0
        self.Svec = np.linspace(self.Smin, self.Smax, self.Ns+1)
        self.Tvec = np.linspace(0, T, self.Nt+1)
        self.grid = np.zeros(shape=(self.Ns+1, self.Nt+1))
    
    def _set_terminal_condition_(self):
        pass
    
    def _set_boundary_condition_(self):
        tau = self.Tvec[-1] - self.Tvec     
        self.grid[0,  :] = self.lR * np.exp(-self.r*tau*self.lRFlag)
        self.grid[-1, :] = self.hR * np.exp(-self.r*tau*self.hRFlag)
        
    def _set_coefficient_(self):
        drift = (self.r-self.q)*self.Svec[1:-1]/self.dS
        diffusion_square = (self.sigma*self.Svec[1:-1]/self.dS)**2
        
        self.l = 0.5*(diffusion_square - drift)
        self.c = -diffusion_square - self.r
        self.u = 0.5*(diffusion_square + drift)
        
    def _solve_(self):
        pass
    
    def _interpolate_(self):
        tck = spi.splrep( self.Svec, self.grid[:,0], k=3 )
        return spi.splev( self.S, tck )
    
    def price(self):
        self._set_terminal_condition_()
        self._set_boundary_condition_()
        self._set_coefficient_()
        self._set_matrix_()
        self._solve_()
        return self._interpolate_()

In [22]:
class DOT_CN(DOT_FD):

    theta = 0.5
    
    def _set_matrix_(self):
        self.A = sp.diags([self.l[1:], self.c, self.u[:-1]], [-1, 0, 1],  format='csc')
        self.I = sp.eye(self.Ns-1)
        self.M1 = self.I + (1-self.theta)*self.dt*self.A
        self.M2 = self.I - self.theta*self.dt*self.A
    
    def _solve_(self):           
        _, M_lower, M_upper = sla.lu(self.M2.toarray())        
        for j in reversed(np.arange(self.Nt)):
            
            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )

测试四个 DOT
- 第一个是触碰上下障碍都是 PaH (`RFlag=[0,0]`)
- 第二个是触碰上障碍是 PaE 下障碍是 PaH (`RFlag=[1,0]`)
- 第三个是触碰上障碍是 PaH 下障碍是 PaE (`RFlag=[0,1]`)
- 第四个触碰上下障碍是 PaE (`RFlag=[1,1]`)

In [23]:
(Ns, Nt ) = (500, np.ceil(T*360))

In [24]:
(optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag) = ('DOT', 100, 1, 0.025, 0, 0.1, 80, 103, [2,2], [0,0])
DOT1 = DOT_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag, Ns, Nt )
DOT2 = DOT_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R, [1,0], Ns, Nt )
DOT3 = DOT_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R, [0,1], Ns, Nt )
DOT4 = DOT_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R, [1,1], Ns, Nt )

V_DOT1 = DOT1.price()
V_DOT2 = DOT2.price()
V_DOT3 = DOT3.price()
V_DOT4 = DOT4.price()

print( f'DOT 上 PaH 下 PaH = {V_DOT1:.4f}' )
print( f'DOT 上 PaE 下 PaH = {V_DOT2:.4f}' )
print( f'DOT 上 PaH 下 PaE = {V_DOT3:.4f}' )
print( f'DOT 上 PaE 下 PaE = {V_DOT4:.4f}' )

DOT 上 PaH 下 PaH = 1.6394
DOT 上 PaE 下 PaH = 1.6393
DOT 上 PaH 下 PaE = 1.6077
DOT 上 PaE 下 PaE = 1.6075


再对比 DOT 和 OTU，发现 DOT PaH 价值高于 OTU PaH， DOT PaE 价值高于 OTU PaE，这个很好理解，DOT 比 OTU 能多触碰下障碍而获得返还，因此价值更高。

In [25]:
print( f'OTU PaH = {V_OTU1:.4f}' )
print( f'OTU PaE = {V_OTU2 :.4f}' )

OTU PaH = 1.6136
OTU PaE = 1.5818


In [26]:
d = {'DOT PaH':V_DOT1, 'DOT PaH/PaE':V_DOT2, 'OTU PaH':V_OTU1, 'DOT PaE/PaH':V_DOT3, 'DOT PaE':V_DOT4, 'OTU PaE':V_OTU2}
s = pd.Series(d)
s.iplot( xTitle='期权种类', yTitle='$', mode='lines+markers', size=10, 
         title='OTU/DOT (80%, 103%)', theme='space' )

In [27]:
d = {'DOT':[V_DOT1, V_DOT2, V_DOT3, V_DOT4], 'OTU':[V_OTU1, np.nan, np.nan, V_OTU2]}
s = pd.DataFrame(d, index=['PaH','PaE/PaH','PaH/PaE','PaE'])
s.iplot( xTitle='返还种类', yTitle='$', mode='lines+markers', size=10, 
         title='OTU/DOT (80%, 103%)', theme='space' )

### <font color='#2b4750' face='微软雅黑'>3.2 普通 DNT</font><a name='3.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

用平价公式推出 DNT 的价值，对于 DNT，返还支付方式只能是 PaE，因此 RFlag 设置为 1 (PaE) 而不是 0 (PaH)。此外只有在上下障碍都不触碰的情况下才有返还，因此只需设定一个返还值。

In [28]:
(optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag) = ('DNT', 100, 1, 0.025, 0, 0.1, 80, 103, 2, 1)

**第一步 - 计算相对应 DOT 期权价值**

In [29]:
optionFlag = str.replace(optionFlag, 'N', 'O')
DOT = DOT_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, [R,R], [RFlag,RFlag], Ns, Nt )
V_DOT = DOT.price()
print(V_DOT)

1.6075064975448607


**第二步 - 计算返还现值**

In [30]:
V_R = R*np.exp(-r*T)
V_R

1.9506198240566652

**第三步 - 根据平价公式计算 DNT 价值**

In [559]:
V_DNT = V_R - V_DOT
V_DNT

0.3431133265118045

### <font color='#2b4750' face='微软雅黑'>3.3 变种 DOT</font><a name='3.3'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

雪球产品第二部分的变种 DNT 和普通 DNT 情况相比，不同点在于上障碍是离散的，每个月检查是否敲出，但下障碍是连续，每天检查是否敲出。

由于是上障碍是离散的，那么上边界不再是 $L_h$，而是要延申到 $4\times\text{max}(S_0,L_h)$：

<img src="DOT discrete.png" width=50% height=50%>

In [31]:
class DoubleOneTouchOptionV():
    
    def __init__(self, optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag, KO_freq=None, T_KO=None):
        self.S = S
        self.Ll = Ll
        self.Lh = Lh
        self.r = r
        self.q = q
        self.T = T
        self.sigma = sigma
        self.lR = R[0]
        self.hR = R[1]
        self.lPaE = RFlag[0]
        self.hPaE = RFlag[1]
        self.lRFlag = 1 if self.lPaE else 0
        self.hRFlag = 1 if self.hPaE else 0
        self.optionFlag = optionFlag
        self.KO_freq = ['daily','daily'] if KO_freq is None else KO_freq
        self.is_down_continuous = self.KO_freq[0]=='daily'
        self.is_up_continuous = self.KO_freq[1]=='daily'
        self.T_KO = [np.array([]), np.array([])] if T_KO is None else T_KO

In [32]:
class DOTV_FD(DoubleOneTouchOptionV):
    
    def __init__(self, optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag, Ns, Nt, KO_freq=None, T_KO=None):
        super().__init__(optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag, KO_freq, T_KO)
                
        self.Smin = self.Ll if self.is_down_continuous else 0 
        self.Smax = self.Lh if self.is_up_continuous else 4*np.maximum(self.S, self.Lh)
        self.Ns = int(Ns)
        self.Nt = int(Nt)    
        self.dS = (self.Smax-self.Smin)/Ns * 1.0
        self.dt = T/Nt*1.0
        self.Svec = np.linspace(self.Smin, self.Smax, self.Ns+1)
        self.Tvec = np.linspace(0, T, self.Nt+1)
        self.grid = np.zeros(shape=(self.Ns+1, self.Nt+1))
    
    def _set_terminal_condition_(self):
        pass
        
    def _set_boundary_condition_(self):
        tau = self.Tvec[-1] - self.Tvec
        lDF = np.exp(-self.r*tau*self.lRFlag)
        hDF = np.exp(-self.r*tau*self.hRFlag)
        
        if (not self.is_down_continuous) and (not np.isscalar(self.lR)): # discrete down barrier with multiple rebates
            f = spi.interp1d(self.T_KO[0], self.lR, kind='next', bounds_error=False, fill_value=(self.lR[0],self.lR[-1]))
            self.grid[0,  :] = f(self.Tvec) * lDF
        else: # discrete down barrier with single rebate, continous down barrier
            self.grid[0,  :] = self.lR * lDF
        
        if (not self.is_up_continuous) and (not np.isscalar(self.hR)): # discrete up barrier with multiple rebates
            f = spi.interp1d(self.T_KO[1], self.hR, kind='next', bounds_error=False, fill_value=(self.hR[0],self.hR[-1]))
            self.grid[-1,  :] = f(self.Tvec) * hDF
        else: # discrete down barrier with single rebate, continous down barrier
            self.grid[-1,  :] = self.hR * hDF
    
    def _set_coefficient_(self):
        drift = (self.r-self.q)*self.Svec[1:-1]/self.dS
        diffusion_square = (self.sigma*self.Svec[1:-1]/self.dS)**2
        
        self.l = 0.5*(diffusion_square - drift)
        self.c = -diffusion_square - self.r
        self.u = 0.5*(diffusion_square + drift)
        
    def _solve_(self):
        pass
    
    def _interpolate_(self):
        tck = spi.splrep( self.Svec, self.grid[:,0], k=3 )
        return spi.splev( self.S, tck )
    
    def price(self):
        self._set_terminal_condition_()
        self._set_boundary_condition_()
        self._set_coefficient_()
        self._set_matrix_()
        self._solve_()
        return self._interpolate_()

In [33]:
class DOTV_CN(DOTV_FD):

    theta = 0.5
    
    def _set_matrix_(self):
        self.A = sp.diags([self.l[1:], self.c, self.u[:-1]], [-1, 0, 1],  format='csc')
        self.I = sp.eye(self.Ns-1)
        self.M1 = self.I + (1-self.theta)*self.dt*self.A
        self.M2 = self.I - self.theta*self.dt*self.A
    
    def _solve_(self):     
        if self.is_down_continuous and self.is_up_continuous:
            self._solve_continuous_()
        else:
            self._solve_discrete_()
    
    def _solve_discrete_(self):     
        _, M_lower, M_upper = sla.lu(self.M2.toarray())
        
        # find the KO index from T vector
        KO_idx_down = np.searchsorted(self.Tvec, self.T_KO[0])
        KO_idx_up = np.searchsorted(self.Tvec, self.T_KO[1])
        
        # update U on [Smin, L_l] if Down KO, update U on [L_h, Smax] if Up KO
        update_down_bool_idx = (self.Svec<=self.Ll)
        update_up_bool_idx = (self.Svec>=self.Lh)
        
        tau_l = self.T - self.T_KO[0]
        tau_h = self.T - self.T_KO[1]
        rbt_l = self.lR * np.exp(-self.r*tau_l*self.lRFlag)
        rbt_h = self.hR * np.exp(-self.r*tau_h*self.hRFlag)
        
        for j in reversed(np.arange(self.Nt)):
            
            if j+1 in KO_idx_down: # Down Knock-out
                k = np.where(KO_idx_down==j+1)[0]
                self.grid[update_down_bool_idx, j+1] = rbt_l[k]
            
            if j+1 in KO_idx_up: # Up Knock-out
                k = np.where(KO_idx_up==j+1)[0]
                self.grid[update_up_bool_idx, j+1] = rbt_h[k]
                
            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )
    
    def _solve_continuous_(self):           
        _, M_lower, M_upper = sla.lu(self.M2.toarray())        
        for j in reversed(np.arange(self.Nt)):
            
            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )

In [34]:
(Ns, Nt ) = (500, np.ceil(T*360))

In [35]:
(optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag) = ('DOT', 100, 1, 0.025, 0, 0.1, 80, 103, [2,2], [0,0])

DOT1 = DOT_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag, Ns, Nt )
DOT2 = DOT_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R, [1,0], Ns, Nt )
DOT3 = DOT_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R, [0,1], Ns, Nt )
DOT4 = DOT_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R, [1,1], Ns, Nt )

V_DOT1 = DOT1.price()
V_DOT2 = DOT2.price()
V_DOT3 = DOT3.price()
V_DOT4 = DOT4.price()

print( f'连续障碍 DOT 上 PaH 下 PaH = {V_DOT1:.4f}' )
print( f'连续障碍 DOT 上 PaE 下 PaH = {V_DOT2:.4f}' )
print( f'连续障碍 DOT 上 PaH 下 PaE = {V_DOT3:.4f}' )
print( f'连续障碍 DOT 上 PaE 下 PaE = {V_DOT4:.4f}' )

连续障碍 DOT 上 PaH 下 PaH = 1.6394
连续障碍 DOT 上 PaE 下 PaH = 1.6393
连续障碍 DOT 上 PaH 下 PaE = 1.6077
连续障碍 DOT 上 PaE 下 PaE = 1.6075


In [36]:
freq = ['daily', 'monthly']
T_KO = np.arange(30, 361, 30)/360
T_DKO = [np.array([]), T_KO]

In [37]:
DOTV1 = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag, Ns, Nt, freq, T_DKO )
DOTV2 = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R, [1,0], Ns, Nt, freq, T_DKO )
DOTV3 = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R, [0,1], Ns, Nt, freq, T_DKO )
DOTV4 = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R, [1,1], Ns, Nt, freq, T_DKO )

V_DOTV1 = DOTV1.price()
V_DOTV2 = DOTV2.price()
V_DOTV3 = DOTV3.price()
V_DOTV4 = DOTV4.price()

print( f'离散障碍 DOT 上 PaH 下 PaH = {V_DOTV1:.4f}' )
print( f'离散障碍 DOT 上 PaE 下 PaH = {V_DOTV2:.4f}' )
print( f'离散障碍 DOT 上 PaH 下 PaE = {V_DOTV3:.4f}' )
print( f'离散障碍 DOT 上 PaE 下 PaE = {V_DOTV4:.4f}' )

离散障碍 DOT 上 PaH 下 PaH = 1.4348
离散障碍 DOT 上 PaE 下 PaH = 1.4347
离散障碍 DOT 上 PaH 下 PaE = 1.4112
离散障碍 DOT 上 PaE 下 PaE = 1.4110


In [38]:
d = {'连续障碍':[V_DOT1, V_DOT2, V_DOT3, V_DOT4], '离散障碍':[V_DOTV1, V_DOTV2, V_DOTV3, V_DOTV4]}
s = pd.DataFrame(d, index=['PaH','PaE/PaH','PaH/PaE','PaE'])
s.iplot( xTitle='返还种类', yTitle='$', mode='lines+markers', size=10, 
         title='DOT 连续/离散 (80%, 103%)', theme='space' )

测试多个返还的情况。

In [39]:
R_vec = [2, 2*R[0]*T_KO]
R_vec

[2,
 array([0.33333333, 0.66666667, 1.        , 1.33333333, 1.66666667,
        2.        , 2.33333333, 2.66666667, 3.        , 3.33333333,
        3.66666667, 4.        ])]

In [40]:
DOTV1 = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R_vec, RFlag, Ns, Nt, freq, T_DKO )
DOTV2 = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R_vec, [1,0], Ns, Nt, freq, T_DKO )
DOTV3 = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R_vec, [0,1], Ns, Nt, freq, T_DKO )
DOTV4 = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R_vec, [1,1], Ns, Nt, freq, T_DKO )

V_DOTV1 = DOTV1.price()
V_DOTV2 = DOTV2.price()
V_DOTV3 = DOTV3.price()
V_DOTV4 = DOTV4.price()

print( f'离散障碍返还从少到多 DOT 上 PaH 下 PaH = {V_DOTV1:.4f}' )
print( f'离散障碍返还从少到多 DOT 上 PaE 下 PaH = {V_DOTV2:.4f}' )
print( f'离散障碍返还从少到多 DOT 上 PaH 下 PaE = {V_DOTV3:.4f}' )
print( f'离散障碍返还从少到多 DOT 上 PaE 下 PaE = {V_DOTV4:.4f}' )

离散障碍返还从少到多 DOT 上 PaH 下 PaH = 0.9324
离散障碍返还从少到多 DOT 上 PaE 下 PaH = 0.9322
离散障碍返还从少到多 DOT 上 PaH 下 PaE = 0.9214
离散障碍返还从少到多 DOT 上 PaE 下 PaE = 0.9213


In [41]:
R_vec = [2, 2*R[0]*T_KO[::-1]]
R_vec

[2,
 array([4.        , 3.66666667, 3.33333333, 3.        , 2.66666667,
        2.33333333, 2.        , 1.66666667, 1.33333333, 1.        ,
        0.66666667, 0.33333333])]

In [42]:
DOTV1 = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R_vec, RFlag, Ns, Nt, freq, T_DKO )
DOTV2 = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R_vec, [1,0], Ns, Nt, freq, T_DKO )
DOTV3 = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R_vec, [0,1], Ns, Nt, freq, T_DKO )
DOTV4 = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, R_vec, [1,1], Ns, Nt, freq, T_DKO )

V_DOTV1r = DOTV1.price()
V_DOTV2r = DOTV2.price()
V_DOTV3r = DOTV3.price()
V_DOTV4r = DOTV4.price()

print( f'离散障碍返还从多到少 DOT 上 PaH 下 PaH = {V_DOTV1r:.4f}' )
print( f'离散障碍返还从多到少 DOT 上 PaE 下 PaH = {V_DOTV2r:.4f}' )
print( f'离散障碍返还从多到少 DOT 上 PaH 下 PaE = {V_DOTV3r:.4f}' )
print( f'离散障碍返还从多到少 DOT 上 PaE 下 PaE = {V_DOTV4r:.4f}' )

离散障碍返还从多到少 DOT 上 PaH 下 PaH = 2.1715
离散障碍返还从多到少 DOT 上 PaE 下 PaH = 2.1713
离散障碍返还从多到少 DOT 上 PaH 下 PaE = 2.1312
离散障碍返还从多到少 DOT 上 PaE 下 PaE = 2.1311


In [43]:
d = {'返还从多到少':[V_DOTV1, V_DOTV2, V_DOTV3, V_DOTV4], '返还从少到多':[V_DOTV1r, V_DOTV2r, V_DOTV3r, V_DOTV4r]}
s = pd.DataFrame(d, index=['PaH','PaE/PaH','PaH/PaE','PaE'])
s.iplot( xTitle='返还种类', yTitle='$', mode='lines+markers', size=10, 
         title='DOT 多返还 (80%, 103%)', theme='space' )

### <font color='#2b4750' face='微软雅黑'>3.4 变种 DNT</font><a name='3.4'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

用平价公式推出 DNT 的价值，对于 DNT，返还支付方式只能是 PaE，因此 RFlag 设置为 1 (PaE) 而不是 0 (PaH)。此外只有在上下障碍都不触碰的情况下才有返还，因此只需设定一个返还值。

In [44]:
(optionFlag, S, T, r, q, sigma, Ll, Lh, R, RFlag) = ('DNT', 100, 1, 0.025, 0, 0.1, 80, 103, 2, 1)

**第一步 - 计算相对应 DOT 期权价值**

In [45]:
optionFlag = str.replace(optionFlag, 'N', 'O')
DOT = DOTV_CN( optionFlag, S, T, r, q, sigma, Ll, Lh, [R,R], [RFlag,RFlag], Ns, Nt, freq, T_DKO )
V_DOT = DOT.price()
print(V_DOT)

1.4110487031707901


**第二步 - 计算返还现值**

In [46]:
V_R = R*np.exp(-r*T)
V_R

1.9506198240566652

**第三步 - 根据平价公式计算 DNT 价值**

In [47]:
V_DNT = V_R - V_DOT
V_DNT

0.5395711208858751

# <font color='#dc2624' face='微软雅黑'>4. 雪球敲入不敲出情况</font><a name='4'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

敲入不敲出时候的支付可由一个**上涨失效看跌障碍期权 (UOP)** 和一个**双边失效看跌障碍期权 (DKOP)**

### <font color='#2b4750' face='微软雅黑'>4.1 变种 UOP</font><a name='4.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#4)

变种 UOP 和普通 UOP 情况相比，不同点在于障碍是离散的，每个月检查是否敲出。正因为是离散障碍，上边界不再是 $L$，而是要延申到 $4\times\text{max}(S_0,L)$。下面两图对比普通 UOP 和变种 UOP 的终值和边界条件。

### <font color='blue' face='微软雅黑'>普通 UOP</font>
<img src="UOP.PNG" width=50% height=50%>

### <font color='blue' face='微软雅黑'>变种 UOP</font>
<img src="UOP discrete.png" width=50% height=50%>

In [48]:
class SingleBarrierKnockOutOption():

    def __init__(self, optionFlag, S, K, T, r, q, sigma, L, R, RFlag, KO_freq=None, T_KO=None):
        self.S = S
        self.K = K
        self.L = L
        self.r = r
        self.q = q
        self.T = T
        self.sigma = sigma
        self.R = R
        self.PaE = RFlag
        self.RFlag = 1 if self.PaE else 0
        self.optionFlag = optionFlag
        self.is_down = (optionFlag[0].lower()=='d')
        self.is_out = (optionFlag[1].lower()=='o')
        self.is_call = (optionFlag[2].lower()=='c')
        self.eta = 1 if self.is_down else -1
        self.phi = 1 if self.is_out else -1
        self.omega = 1 if self.is_call else -1
        self.KO_freq = 'daily' if KO_freq is None else KO_freq
        self.is_continuous = KO_freq is None
        self.T_KO = np.array([]) if T_KO is None else T_KO

In [49]:
class SB_KO_FD(SingleBarrierKnockOutOption):
    
    def __init__(self, optionFlag, S, K, T, r, q, sigma, L, R, RFlag, Ns, Nt, KO_freq=None, T_KO=None):
        super().__init__(optionFlag, S, K, T, r, q, sigma, L, R, RFlag, KO_freq, T_KO)
                
        if self.is_down:
            self.Smin = self.L if self.is_continuous else 0 
            self.Smax = 4*np.maximum(self.S, self.K)
        else:
            self.Smin = 0
            self.Smax = self.L if self.is_continuous else 4*np.maximum(self.S, self.K)
        
        self.Ns = int(Ns)
        self.Nt = int(Nt)    
        self.dS = (self.Smax-self.Smin)/Ns * 1.0
        self.dt = T/Nt*1.0
        self.Svec = np.linspace(self.Smin, self.Smax, self.Ns+1)
        self.Tvec = np.linspace(0, T, self.Nt+1)
        self.grid = np.zeros(shape=(self.Ns+1, self.Nt+1))
        
    def _set_terminal_condition_(self):
        self.grid[:, -1] = np.maximum(self.omega*(self.Svec - self.K), 0)
    
    def _set_boundary_condition_(self):
        tau = self.Tvec[-1] - self.Tvec     
        DFq = np.exp(-self.q*tau)
        DFr = np.exp(-self.r*tau)

        if self.is_down:
            self.grid[0,  :] = self.R * np.exp(-self.r*tau*self.RFlag)
            self.grid[-1, :] = np.maximum(self.omega*(self.Svec[-1]*DFq - self.K*DFr), 0)
        else:
            self.grid[0,  :] = np.maximum(self.omega*(self.Svec[0]*DFq - self.K*DFr), 0)
            self.grid[-1, :] = self.R * np.exp(-self.r*tau*self.RFlag)
            
    def _set_coefficient_(self):
        drift = (self.r-self.q)*self.Svec[1:-1]/self.dS
        diffusion_square = (self.sigma*self.Svec[1:-1]/self.dS)**2
        
        self.l = 0.5*(diffusion_square - drift)
        self.c = -diffusion_square - self.r
        self.u = 0.5*(diffusion_square + drift)
        
    def _solve_(self):
        pass
    
    def _interpolate_(self):
        tck = spi.splrep( self.Svec, self.grid[:,0], k=3 )
        return spi.splev( self.S, tck )
    
    def price(self):
        self._set_terminal_condition_()
        self._set_boundary_condition_()
        self._set_coefficient_()
        self._set_matrix_()
        self._solve_()
        return self._interpolate_()

In [50]:
class SB_KO_CN(SB_KO_FD):

    theta = 0.5
    
    def _set_matrix_(self):
        self.A = sp.diags([self.l[1:], self.c, self.u[:-1]], [-1, 0, 1],  format='csc')
        self.I = sp.eye(self.Ns-1)
        self.M1 = self.I + (1-self.theta)*self.dt*self.A
        self.M2 = self.I - self.theta*self.dt*self.A
    
    def _solve_(self):     
        if self.is_continuous:
            self._solve_continuous_()
        else:
            self._solve_discrete_()
    
    def _solve_discrete_(self):    
        _, M_lower, M_upper = sla.lu(self.M2.toarray())
        
        # find the KO index from T vector
        KO_idx = np.searchsorted(self.Tvec, self.T_KO)
        
        # update U on [Smin, L] if Down KO, update U on [L, Smax] if Up KO
        update_bool_idx = (self.Svec<=self.L) if self.is_down else (self.Svec>=self.L)
        
        tau = self.T - self.T_KO
        rbt = self.R * np.exp(-self.r*tau*self.RFlag)
                
        for j in reversed(np.arange(self.Nt)):
            
            if j+1 in KO_idx: # Knock-out
                k = np.where(KO_idx==j+1)[0]
                self.grid[update_bool_idx, j+1] = rbt[k]  

            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )
    
    def _solve_continuous_(self):     
        _, M_lower, M_upper = sla.lu(self.M2.toarray())
        for j in reversed(np.arange(self.Nt)):
            
            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )

### <font color='#2b4750' face='微软雅黑'>4.2 变种 DKOP</font><a name='4.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#4)

变种 DKOP 和普通 DKOP 情况相比，不同点在于上障碍是离散的，每个月检查是否敲出。正因为是离散上障碍，上边界不再是 $L$，而是要延申到 $4\times\text{max}(S_0,L)$。下面两图对比普通 DKOP 和变种 DKOP 的终值和边界条件。

### <font color='blue' face='微软雅黑'>普通 DKOP</font>
<img src="DKOP.PNG" width=50% height=50%>

### <font color='blue' face='微软雅黑'>变种 DKOP</font>
<img src="DKOP discrete.PNG" width=50% height=50%>

In [51]:
class DoubleBarrierKnockOutOption():
    
    def __init__(self, optionFlag, S, K, T, r, q, sigma, Ll, Lh, R, RFlag, KO_freq=None, T_KO=None):
        self.S = S
        self.K = K
        self.Ll = Ll
        self.Lh = Lh
        self.r = r
        self.q = q
        self.T = T
        self.sigma = sigma
        self.lR = R[0]
        self.hR = R[1]
        self.lPaE = RFlag[0]
        self.hPaE = RFlag[1]
        self.lRFlag = 1 if self.lPaE else 0
        self.hRFlag = 1 if self.hPaE else 0
        self.optionFlag = optionFlag
        self.is_out = (optionFlag[2].lower()=='o')
        self.is_call = (optionFlag[-1].lower()=='c')
        self.phi = 1 if self.is_out else -1
        self.omega = 1 if self.is_call else -1
        self.KO_freq = ['daily','daily'] if KO_freq is None else KO_freq
        self.is_down_continuous = self.KO_freq[0]=='daily'
        self.is_up_continuous = self.KO_freq[1]=='daily'
        self.T_KO = [np.array([]), np.array([])] if T_KO is None else T_KO

In [52]:
class DB_KO_FD(DoubleBarrierKnockOutOption):
    
    def __init__(self, optionFlag, S, K, T, r, q, sigma, Ll, Lh, R, RFlag, Ns, Nt, freq=None, T_KO=None):
        super().__init__(optionFlag, S, K, T, r, q, sigma, Ll, Lh, R, RFlag, freq, T_KO)
        
        self.Smin = self.Ll if self.is_down_continuous else 0 
        self.Smax = self.Lh if self.is_up_continuous else 4*np.maximum(self.S, self.K)

        self.Ns = int(Ns)
        self.Nt = int(Nt)    
        self.dS = (self.Smax-self.Smin)/Ns * 1.0
        self.dt = T/Nt*1.0
        self.Svec = np.linspace(self.Smin, self.Smax, self.Ns+1)
        self.Tvec = np.linspace(0, T, self.Nt+1)
        self.grid = np.zeros(shape=(self.Ns+1, self.Nt+1))
    
    def _set_terminal_condition_(self):
        self.grid[:, -1] = np.maximum(self.omega*(self.Svec - self.K), 0)
    
    def _set_boundary_condition_(self):
        tau = self.Tvec[-1] - self.Tvec
        self.grid[0,  :] = self.lR * np.exp(-self.r*tau*self.lRFlag)
        self.grid[-1, :] = self.hR * np.exp(-self.r*tau*self.hRFlag)
        
    def _set_coefficient_(self):
        drift = (self.r-self.q)*self.Svec[1:-1]/self.dS
        diffusion_square = (self.sigma*self.Svec[1:-1]/self.dS)**2
        
        self.l = 0.5*(diffusion_square - drift)
        self.c = -diffusion_square - self.r
        self.u = 0.5*(diffusion_square + drift)
        
    def _solve_(self):
        pass
    
    def _interpolate_(self):
        tck = spi.splrep( self.Svec, self.grid[:,0], k=3 )
        return spi.splev( self.S, tck )
    
    def price(self):
        self._set_terminal_condition_()
        self._set_boundary_condition_()
        self._set_coefficient_()
        self._set_matrix_()
        self._solve_()
        return self._interpolate_()

In [53]:
class DB_KO_CN(DB_KO_FD):

    theta = 0.5
    
    def _set_matrix_(self):
        self.A = sp.diags([self.l[1:], self.c, self.u[:-1]], [-1, 0, 1],  format='csc')
        self.I = sp.eye(self.Ns-1)
        self.M1 = self.I + (1-self.theta)*self.dt*self.A
        self.M2 = self.I - self.theta*self.dt*self.A

    def _solve_(self):     
        if self.is_down_continuous and self.is_up_continuous:
            self._solve_continuous_()
        else:
            self._solve_discrete_()
    
    def _solve_discrete_(self):     
        _, M_lower, M_upper = sla.lu(self.M2.toarray())
        
        # find the KO index from T vector
        KO_idx_down = np.searchsorted(self.Tvec, self.T_KO[0])
        KO_idx_up = np.searchsorted(self.Tvec, self.T_KO[1])
        
        # update U on [Smin, L_l] if Down KO, update U on [L_h, Smax] if Up KO
        update_down_bool_idx = (self.Svec<=self.Ll)
        update_up_bool_idx = (self.Svec>=self.Lh)
        
        tau_l = self.T - self.T_KO[0]
        tau_h = self.T - self.T_KO[1]
        rbt_l = self.lR * np.exp(-self.r*tau_l*self.lRFlag)
        rbt_h = self.hR * np.exp(-self.r*tau_h*self.hRFlag)
        
        for j in reversed(np.arange(self.Nt)):
            
            if j+1 in KO_idx_down: # Down Knock-out
                k = np.where(KO_idx_down==j+1)[0]
                self.grid[update_down_bool_idx, j+1] = rbt_l[k]
            
            if j+1 in KO_idx_up: # Up Knock-out
                k = np.where(KO_idx_up==j+1)[0]
                self.grid[update_up_bool_idx, j+1] = rbt_h[k]
                
            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )
    
    def _solve_continuous_(self):     
        _, M_lower, M_upper = sla.lu(self.M2.toarray())
        for j in reversed(np.arange(self.Nt)):
            
            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )

### <font color='#2b4750' face='微软雅黑'>4.3 变种 KIKO</font><a name='4.3'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#4)

雪球第三部分的 KIKO 信息如下两表所示：

|产品特征|股票价格|行权价格|期权类别|连续利率|连续红利率|波动率|到期年限|
|---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
||100|100|看跌|2.5%|0%|10%|1|

|AutoCall结构|敲入价|敲入频率|敲出价|敲出频率|
|---|:---:|:---:|:---:|:---:|
||80%|日观察|103%|月观察|

In [54]:
(S, K, r, q, sigma, T, l, u) = (100, 100, 0.025, 0, 0.1, 1, 0.8, 1.03)
(L, U) = (l*S, u*S)
(KI_flag, KI_freq, KO_flag, KO_freq, opt_flag, first_obs_idx) = ('lower','daily','upper','monthly','put',0)
T = T - first_obs_idx/360
T_KO = np.arange(30, 361, 30)/360

In [55]:
isKIdown = (KI_flag.lower()=='lower')
isPut = (opt_flag.lower()=='put')

# When KI lower barrier, V3 = UOP - DKOP, or V3 = UOC - DKOC
# When KI upper barrier, V3 = DOP - DKOP, or V3 = DOC - DKOC

SBOFlag = 'UOP' if isPut else 'UOC' if isKIdown else 'DOP' if isPut else 'DOC'
DBOFlag = 'DKOP' if isPut else 'DKOC'
freq = [KI_freq, KO_freq] if isKIdown else [KO_freq, KI_freq]
T_DKO = [np.array([]), T_KO] if isKIdown else [T_KO, np.array([])]

In [56]:
(Ns, Nt) = (500, np.ceil(T*360))

In [57]:
UOP_d = SB_KO_CN( SBOFlag, S, K, T, r, q, sigma, U, 0, 0, Ns, Nt, KO_freq, T_KO )
V_UOP_d = UOP_d.price()

UOP_c = SB_KO_CN( SBOFlag, S, K, T, r, q, sigma, U, 0, 0, Ns, Nt)
V_UOP_c = UOP_c.price()

print( f'离散障碍 UOP = {V_UOP_d:.4f}' )
print( f'连续障碍 UOP = {V_UOP_c:.4f}' )

离散障碍 UOP = 2.0559
连续障碍 UOP = 1.5482


In [58]:
DKOP_d = DB_KO_CN( DBOFlag, S, K, T, r, q, sigma, L, U, [0,0], [0,0], Ns, Nt, freq, T_DKO )
V_DKOP_d = DKOP_d.price()

DKOP_c = DB_KO_CN( DBOFlag, S, K, T, r, q, sigma, L, U, [0,0], [0,0], Ns, Nt )
V_DKOP_c = DKOP_c.price()

print( f'离散障碍 DKOP = {V_DKOP_d:.4f}' )
print( f'连续障碍 DKOP = {V_DKOP_c:.4f}' )

离散障碍 DKOP = 1.7467
连续障碍 DKOP = 1.2975


KIKO 可由一个 UOP 和一个 DKOP 来复制，回顾公式

<br>
<font color='blue'>
\begin{equation}
V_3 = \frac{1}{S_0}\times[\text{DKOP}(K=S_0,L_u,L_l) - \text{UOP}(K=S_0,L_u)]
\end{equation}
</font>

In [59]:
V3 = (V_DKOP_d - V_UOP_d)/S
print(f'UOP = {V_UOP_d:.4f}')
print(f'DKOP = {V_DKOP_d:.4f}')
print(f'KIKO = {V3:.4f}')

UOP = 2.0559
DKOP = 1.7467
KIKO = -0.0031


UOP 和 DKOP 都应该比普通的 put 便宜，验证一下。先写出欧式期权有限差分的代码。

### <font color='blue' face='微软雅黑'>欧式期权</font>

In [60]:
class EuropeanOption():
    
    def __init__(self, S, K, r, q, T, sigma, option_type):
        self.S = S
        self.K = K
        self.r = r
        self.q = q
        self.T = T
        self.sigma = sigma
        self.option_type = option_type
        self.is_call = (option_type[0].lower()=='c')
        self.omega = 1 if self.is_call else -1

In [61]:
class EO_FD(EuropeanOption):
    
    def __init__(self, S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt):
        super().__init__(S, K, r, q, T, sigma, option_type)
        self.Smin = Smin
        self.Smax = Smax
        self.Ns = int(Ns)
        self.Nt = int(Nt)
        self.dS = (Smax-Smin)/Ns * 1.0
        self.dt = T/Nt*1.0
        self.Svec = np.linspace(Smin, Smax, self.Ns+1)
        self.Tvec = np.linspace(0, T, self.Nt+1)
        self.grid = np.zeros(shape=(self.Ns+1, self.Nt+1))
        
    def _set_terminal_condition_(self):
        self.grid[:, -1] = np.maximum(self.omega*(self.Svec - self.K), 0)
    
    def _set_boundary_condition_(self):
        tau = self.Tvec[-1] - self.Tvec;     
        DFq = np.exp(-q*tau)
        DFr = np.exp(-r*tau);

        self.grid[0,  :] = np.maximum(self.omega*(self.Svec[0]*DFq - self.K*DFr), 0)
        self.grid[-1, :] = np.maximum(self.omega*(self.Svec[-1]*DFq - self.K*DFr), 0)        
        
    def _set_coefficient_(self):
        drift = (self.r-self.q)*self.Svec[1:-1]/self.dS
        diffusion_square = (self.sigma*self.Svec[1:-1]/self.dS)**2
        
        self.l = 0.5*(diffusion_square - drift)
        self.c = -diffusion_square - self.r
        self.u = 0.5*(diffusion_square + drift)
        
    def _solve_(self):
        pass
    
    def _interpolate_(self):
        tck = spi.splrep( self.Svec, self.grid[:,0], k=3 )
        return spi.splev( self.S, tck )
    
    def price(self):
        self._set_terminal_condition_()
        self._set_boundary_condition_()
        self._set_coefficient_()
        self._set_matrix_()
        self._solve_()
        return self._interpolate_()

In [62]:
class EO_CN(EO_FD):

    theta = 0.5
    
    def _set_matrix_(self):
        self.A = sp.diags([self.l[1:], self.c, self.u[:-1]], [-1, 0, 1],  format='csc')
        self.I = sp.eye(self.Ns-1)
        self.M1 = self.I + (1-self.theta)*self.dt*self.A
        self.M2 = self.I - self.theta*self.dt*self.A
    
    def _solve_(self):           
        _, M_lower, M_upper = sla.lu(self.M2.toarray())        
        for j in reversed(np.arange(self.Nt)):
            
            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )

In [63]:
(Smin, Smax) = (0, 4*np.maximum(S,K))
Euro = EO_CN(S, K, r, q, T, sigma, opt_flag, Smin, Smax, Ns, Nt)

V_euro = Euro.price()
print( f'欧式 Euro = {V_euro:.4f}' )

欧式 Euro = 2.8231


In [64]:
d = {'Put':V_euro, 'UOP (离散障碍)':V_UOP_d, 'DKOP (离散障碍)':V_DKOP_d, 'UOP (连续障碍)':V_UOP_c, 'DKOP (连续障碍)':V_DKOP_c}
s = pd.Series(d)

In [65]:
s.iplot( xTitle='种类', yTitle='$', mode='lines+markers', size=10, 
         title='单双障碍 (离散连续) 的期权定价', theme='space' )

Put > 离散 UOP > 离散 DKOP > 连续 UOP > 连续 DKOP，结果合理。  

# <font color='#dc2624' face='微软雅黑'>5. 定价雪球</font><a name='5'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

该雪球结构期权的定价公式如下：

<br>
<font color='blue'>
\begin{align*}
V_{\text{snowball}} &= V_1 + V_2 + V_3 \\
    & = \text{OTU}\left(R=\left[\frac{c_1\%}{12},\frac{2c_1\%}{12},\cdots,\frac{24c_1\%}{12}\right]\right)  \\
    & + \text{DNT}(R=2c_2\%) \\
    & + \frac{1}{S_0}\times[\text{DKOP}(K=S_0,L_u,L_l) - \text{UOP}(K=S_0,L_u)]
\end{align*}
</font>

在接下来三节分别对 $V_1$, $V_2$ 和 $V_3$ 用偏微分方程有限差分来对其定价。

|名称|本金|期初价格|上界|下界|敲出收益(%)|红利收益(%)|到日期|标的价格|到期价格|
|---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|符号|$N$|$S_0$|$L_h$|$L_l$|$c_1$|$c_2$|$T$|$S_t$|$S_T$|
|值|$1$亿|$5000$|$5150$|$4000$|$20$|$20$|$2$|||

上障碍都是离散型的，而下障碍是连续型的。

假设雪球的标的是观察中证 500 从 2019 年 12 月 9 日开始，对应的指数是 5000 点，根据历史两年数据计算出其收益率波动率为 20%，即 <font color='blue'>$\sigma = 20\%$ </font>，从 SHIBOR 3M 曲线上插值出两年时点的零息利率 3%，即 <font color='blue'>$r = 3\%$ </font>。

In [67]:
# 产品特征
(N, S, T, l, u, c1, c2) = (10**8, 5000, 2, 0.8, 1.03, 0.2, 0.2)
(KI_flag, KI_freq, KO_flag, KO_freq) = ('lower','daily','upper','monthly')
(Ll, Lh) = (l*S, u*S)
T_KO = np.arange(1/12, T+0.01, 1/12)

# 从市场数据计算
(r, q, sigma) = (0.03, 0, 0.2)

# 有限差分参数
(Ns, Nt) = (500, np.ceil(T*360))

### <font color='blue' face='微软雅黑'>计算 V1</font>

In [68]:
R_vec = c1*T_KO
OTU = OTV_CN( 'OTU', S, T, r, q, sigma, Ll, R_vec, RFlag, Ns, Nt, KO_freq, T_KO )
V1 = OTU.price() * N
V1

1569813.9848501377

### <font color='blue' face='微软雅黑'>计算 V2</font>

In [69]:
isKIdown = (KI_flag.lower()=='lower')
freq = [KI_freq, KO_freq] if isKIdown else [KO_freq, KI_freq]
T_DKO = [np.array([]), T_KO] if isKIdown else [T_KO, np.array([])]

(R, RFlag) = ([T*c2,T*c2], [1,1])

DOT = DOTV_CN( 'DOT', S, T, r, q, sigma, Ll, Lh, R, RFlag, Ns, Nt, freq, T_DKO )
V_R = (T*c2)*np.exp(-r*T)
DNT = V_R - DOT.price()
V2 = DNT * N
V2

240103.559755922

### <font color='blue' face='微软雅黑'>计算 V3</font>

In [70]:
(opt_flag, K) = ('put', S)
isPut = (opt_flag.lower()=='put')
SBOFlag = 'UOP' if isPut else 'UOC' if isKIdown else 'DOP' if isPut else 'DOC'
DBOFlag = 'DKOP' if isPut else 'DKOC'
freq = [KI_freq, KO_freq] if isKIdown else [KO_freq, KI_freq]
T_DKO = [np.array([]), T_KO] if isKIdown else [T_KO, np.array([])]

UOP = SB_KO_CN( SBOFlag, S, K, T, r, q, sigma, Lh, 0, 0, Ns, Nt, KO_freq, T_KO )
DKOP = DB_KO_CN( DBOFlag, S, K, T, r, q, sigma, Ll, Lh, [0,0], [0,0], Ns, Nt, freq, T_DKO )

V3 = (DKOP.price() - UOP.price()) / S * N
V3

-3649514.3412733166

### <font color='blue' face='微软雅黑'>汇总得到雪球</font>

In [71]:
V_base = V1 + V2 + V3

print(f'敲出部分的价值:        {V1:,.0f} 人民币')
print(f'不敲出不敲入部分的价值：{V2:,.0f} 人民币')
print(f'敲入不敲出部分的价值：  {V3:,.0f} 人民币')
print(f'雪球的价值：           {V_base:,.0f} 人民币')

敲出部分的价值:        1,569,814 人民币
不敲出不敲入部分的价值：240,104 人民币
敲入不敲出部分的价值：  -3,649,514 人民币
雪球的价值：           -1,839,597 人民币


### 如果真实波动率变<font color='blue'>小</font>呢？

In [72]:
sigma = 0.1

OTU = OTV_CN( 'OTU', S, T, r, q, sigma, Ll, R_vec, RFlag, Ns, Nt, KO_freq, T_KO )
V1 = OTU.price() * N

DOT = DOTV_CN( 'DOT', S, T, r, q, sigma, Ll, Lh, R, RFlag, Ns, Nt, freq, T_DKO )
V_R = (T*c2)*np.exp(-r*T)
DNT = V_R - DOT.price()
V2 = DNT * N

UOP = SB_KO_CN( SBOFlag, S, K, T, r, q, sigma, Lh, 0, 0, Ns, Nt, KO_freq, T_KO )
DKOP = DB_KO_CN( DBOFlag, S, K, T, r, q, sigma, Ll, Lh, [0,0], [0,0], Ns, Nt, freq, T_DKO )
V3 = (DKOP.price() - UOP.price()) / S * N

V_mild = V1 + V2 + V3

print(f'敲出部分的价值:        {V1:,.0f} 人民币')
print(f'不敲出不敲入部分的价值：{V2:,.0f} 人民币')
print(f'敲入不敲出部分的价值：  {V3:,.0f} 人民币')
print(f'雪球的价值：           {V_mild:,.0f} 人民币')

敲出部分的价值:        1,569,608 人民币
不敲出不敲入部分的价值：4,728,981 人民币
敲入不敲出部分的价值：  -848,158 人民币
雪球的价值：           5,450,431 人民币


### 如果真实波动率变<font color='blue'>大</font>呢？

In [73]:
sigma = 0.3

OTU = OTV_CN( 'OTU', S, T, r, q, sigma, Ll, R_vec, RFlag, Ns, Nt, KO_freq, T_KO )
V1 = OTU.price() * N

DOT = DOTV_CN( 'DOT', S, T, r, q, sigma, Ll, Lh, R, RFlag, Ns, Nt, freq, T_DKO )
V_R = (T*c2)*np.exp(-r*T)
DNT = V_R - DOT.price()
V2 = DNT * N

UOP = SB_KO_CN( SBOFlag, S, K, T, r, q, sigma, Lh, 0, 0, Ns, Nt, KO_freq, T_KO )
DKOP = DB_KO_CN( DBOFlag, S, K, T, r, q, sigma, Ll, Lh, [0,0], [0,0], Ns, Nt, freq, T_DKO )
V3 = (DKOP.price() - UOP.price()) / S * N

V_wild = V1 + V2 + V3

print(f'敲出部分的价值:        {V1:,.0f} 人民币')
print(f'不敲出不敲入部分的价值：{V2:,.0f} 人民币')
print(f'敲入不敲出部分的价值：  {V3:,.0f} 人民币')
print(f'雪球的价值：           {V_wild:,.0f} 人民币')

敲出部分的价值:        1,590,109 人民币
不敲出不敲入部分的价值：2,337 人民币
敲入不敲出部分的价值：  -5,543,038 人民币
雪球的价值：           -3,950,592 人民币


In [74]:
d = {'较小波动 (10%)':V_mild, '正常波动 (20%)':V_base, '较大波动 (30%)':V_wild}
s = pd.Series(d)

s.iplot( xTitle='波动程度', yTitle='人民币', mode='lines+markers', size=10, 
         title='中证 500 不同波动下的雪球收益 (一亿人民币本金)', theme='polar' )